In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq
api_key=os.getenv("GROQ_APIKEY")
llm=ChatGroq(api_key=api_key,model_name="meta-llama/llama-4-scout-17b-16e-instruct")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F65EF30D10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F65F06CF50>, model_name='meta-llama/llama-4-scout-17b-16e-instruct', model_kwargs={}, groq_api_key=SecretStr('**********'))

Stuff Documents

In [3]:
from langchain.chains import LLMChain
from langchain import PromptTemplate
from langchain_community.document_loaders import PyMuPDFLoader
loader=PyMuPDFLoader("apjspeech.pdf")
docs=loader.load_and_split()
docs

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'source': 'apjspeech.pdf', 'file_path': 'apjspeech.pdf', 'total_pages': 7, 'format': 'PDF 1.3', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'subject': '', 'keywords': '', 'moddate': 'D:20070730160943', 'trapped': '', 'modDate': 'D:20070730160943', 'creationDate': 'D:20070730160943', 'page': 0}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narrate, how I \nenjoyed every minute of my tenure enriched by the wonderful association from each one \nof you, hailing from different walks of life, be it politics, science and technology, \nacademics, arts, literature, business, judiciary, administration,

In [4]:
template=''' 
Write a concise and a short summary of the following speech: {text}
'''
prompt=PromptTemplate(input_variables=['text'],template=template)


In [5]:
from langchain.chains.summarize import load_summarize_chain


In [ ]:
chain=load_summarize_chain(llm,chain_type='stuff',prompt=prompt,verbose=True)
output_summary=chain.run(docs)
output_summary  

C:\Users\DeLL\AppData\Local\Temp\ipykernel_12064\2841055168.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output_summary=chain.run(docs)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
Write a concise and a short summary of the following speech: A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narrate, how I 
enjoyed every minute of my tenure enriched by the wonderful association from each one 
of you, hailing from different walks of life, be it politics, science and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and student 
community who are the future wealth of our country. During my interaction at 
Rashtrapati Bhavan in Delhi and at every state and union territory as well as through my 
online interactions, 

'Here is a concise and short summary of A.P.J. Abdul Kalam\'s departing speech:\n\n**Summary:**\n\nFormer President A.P.J. Abdul Kalam reflects on his five-year tenure and shares key messages and experiences. He emphasizes the need to:\n\n1. Accelerate development to meet the aspirations of the youth.\n2. Empower villages and provide physical connectivity.\n3. Mobilize rural core competence for competitiveness.\n4. Focus on agriculture growth through the "Seed to Food" approach.\n5. Overcome problems through partnership and courage.\n\nHe also highlights the importance of:\n\n* Connectivity for societal transformation\n* Defending the nation with pride\n* A youth movement for a developed India by 2020\n\n**Key Takeaway:**\n\nKalam expresses his confidence in India\'s potential to become a developed nation before 2020, driven by the country\'s natural resources, hardworking people, and the power of its 540 million youth. He urges citizens to work together to realize this vision and emph

MAP REDUCE


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [8]:
docs

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'source': 'apjspeech.pdf', 'file_path': 'apjspeech.pdf', 'total_pages': 7, 'format': 'PDF 1.3', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'subject': '', 'keywords': '', 'moddate': 'D:20070730160943', 'trapped': '', 'modDate': 'D:20070730160943', 'creationDate': 'D:20070730160943', 'page': 0}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narrate, how I \nenjoyed every minute of my tenure enriched by the wonderful association from each one \nof you, hailing from different walks of life, be it politics, science and technology, \nacademics, arts, literature, business, judiciary, administration,

In [9]:
final_documents=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=100).split_documents(docs)
final_documents

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'source': 'apjspeech.pdf', 'file_path': 'apjspeech.pdf', 'total_pages': 7, 'format': 'PDF 1.3', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'subject': '', 'keywords': '', 'moddate': 'D:20070730160943', 'trapped': '', 'modDate': 'D:20070730160943', 'creationDate': 'D:20070730160943', 'page': 0}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narrate, how I \nenjoyed every minute of my tenure enriched by the wonderful association from each one \nof you, hailing from different walks of life, be it politics, science and technology, \nacademics, arts, literature, business, judiciary, administration,

In [10]:
len(final_documents)

13

In [11]:
chunk_prompt='''
Please summarize the below speech:
Speech:'{text}'
Summary:
'''
map_prompt=PromptTemplate(input_variable=['text'],template=chunk_prompt)

In [12]:
final_prompt='''
Provide a final summary of the entire speech with these important points.
Add a motivation title and start the precise summary with an introduction.
Speech:{text}
'''
final_prompt_temp=PromptTemplate(input_variables=['text'],template=final_prompt)
final_prompt_temp

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nProvide a final summary of the entire speech with these important points.\nAdd a motivation title and start the precise summary with an introduction.\nSpeech:{text}\n')

In [13]:
summary_chain=load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=final_prompt_temp,
    verbose=True
)
output=summary_chain.run(final_documents)
output



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarize the below speech:
Speech:'A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narrate, how I 
enjoyed every minute of my tenure enriched by the wonderful association from each one 
of you, hailing from different walks of life, be it politics, science and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and student 
community who are the future wealth of our country. During my interaction at 
Rashtrapati Bhavan in Delhi and at every state and union territory as well as through my 
online interactions, I have many uni


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\DeLL\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\DeLL\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\DeLL\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernelapp.py", lin

RuntimeError: Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject